## `faulthandler`

> **<p style="color:green">!! Note !!</p>**
> <hr>
> This module was added in Python 3.3 and also it is not present in Python 2.x series, but they can download it from https://pypi.org/project/faulthandler/

`faulthandler` can be used to dump Python tracebacks explicitly on 
- a fault, 
- a timeout, 
- a user signal. 

`faulthandler.enable()` installs fault handlers for the `SIGSEGV`, `SIGFPE`, `SIGABRT`, `SIGBUS`, and `SIGILL`signals.
It can also be enabled by setting the `PYTHONFAULTHANDLER=1` environment variable or by using the `-X faulthandler` command line option.

### Dump

`dump_tractback` funtion can be used to dump the `trackback` to a file as shown in the below exmaple. Its syntax is as follows

```python
faulthandler.dump_traceback(file=sys.stderr, all_threads=True)
```

In [1]:
import faulthandler 

faulthandler.enable()

try:
    a = 1/0
except Exception as e:
    with open("Error.log", "a+") as f:
        faulthandler.dump_traceback(file=f, all_threads=True)

### `enable()`

`enable` not only starts the `faulthandler` but can also be used to define the log files as shown in the below example. After executing the below example, SEGFAULT details are 

```python
# Reference: https://codegolf.stackexchange.com/q/22352/38159
# Code by `Gareth Rees` https://codegolf.stackexchange.com/users/2186/gareth-rees

import faulthandler as fh

f = open("Error_auto.log", "a+")
fh.enable(file=f, all_threads=True)

exec(type((lambda:0).__code__)(0,1,0,0,0,b'',(),(),(),'','',1,b''))
```

### `disable()`

It disables the fault handler, thus uninstalls any signal handlers installed by `enable()`.

If it is able to disable fault handler then it returns `True` else returns `False` which can happen if fault handler is already disabled.

### `is_enabled()`

It returns `True` if fault handler is enabled else returns `False`

In [2]:
import faulthandler as fh

print(fh.is_enabled())
print(fh.disable())

fh.enable(all_threads=True)
print(fh.is_enabled())
print(fh.disable())

True
True
True
True


### Examples:

As stated above, we can initiate traceback dump in three scenarios. We are going to cover all of them. 

#### Fatal Errors

`faulthandler.enable()` & `dump_traceback` can be used in these cases as shown in the below examples.

In [3]:
import faulthandler 


faulthandler.enable()
try:
    a = 1/0
except Exception as e:
    with open("Error.log", "a+") as f:
        faulthandler.dump_traceback(file=f, all_threads=True)

### Timeout

`faulthandler.dump_tracebacks_later()` can be used to set the timeout. It will trigger the traceback if any function exceeds to timeout time set in `dump_tracebacks_later` as shown in the below examples.

- Example 1: Default timeout to 5 sec.

```python
import faulthandler


def MySample():
    import time
    time.sleep(10)
    return "Welcome to the Wonders of Python"

faulthandler.dump_traceback_later(5)
MySample()
```

**Output:**
```python
Timeout (0:00:05)!
Thread 0x00007f9b082e7540 (most recent call first):
  File "timeout_example.py", line 10 in MySample
  File "timeout_example.py", line 13 in <module>
```

- Example 2: Default `timeout` to `2 sec` and `repeat = True`.

Use the `repeat=True` option to dump the traceback for each timeout,

```PYTHON
import faulthandler


def MySample():
    import time
    time.sleep(3)
    return "Welcome to the Wonders of Python"


log_file = "dump_later.txt"
f = open(log_file, "w")
faulthandler.dump_traceback_later(2, file=f, repeat=True)
txt = MySample()
print(txt)
f.close()
faulthandler.cancel_dump_traceback_later()

with open(log_file) as fp:
    for line in fp.readlines():
        print(line, end="")
```

**Output:**
```python
Welcome to the Wonders of Python
Timeout (0:00:02)!
Thread 0x00007f9cea59f540 (most recent call first):
  File "timeout_repeat_example.py", line 6 in MySample
  File "timeout_repeat_example.py", line 13 in <module>
```

- Example 3: Default `timeout` to `2 sec`, `repeat = True` and function taking 6 sec to complete.

    In this scenario, the traceback will happen 3 times (6 / 2 = 3) as 

    `log_times = time_taken/timeout`

```python
import faulthandler


def MySample():
    import time
    time.sleep(6)
    return "Welcome to the Wonders of Python"


log_file = "dump_later.txt"
f = open(log_file, "w")
faulthandler.dump_traceback_later(2, file=f, repeat=True)
txt = MySample()
print(txt)
f.close()
faulthandler.cancel_dump_traceback_later()

with open(log_file) as fp:
    for line in fp.readlines():
        print(line, end="")
```
**Output:**
```python
Welcome to the Wonders of Python
Timeout (0:00:02)!
Thread 0x00007f3d95bff540 (most recent call first):
  File "timeout_repeat_2_example.py", line 6 in MySample
  File "timeout_repeat_2_example.py", line 13 in <module>
Timeout (0:00:02)!
Thread 0x00007f3d95bff540 (most recent call first):
  File "timeout_repeat_2_example.py", line 6 in MySample
  File "timeout_repeat_2_example.py", line 13 in <module>
Timeout (0:00:02)!
Thread 0x00007f3d95bff540 (most recent call first):
  File "timeout_repeat_2_example.py", line 6 in MySample
  File "timeout_repeat_2_example.py", line 13 in <module>
```

- Example 4: Default `timeout` to `2 sec`, `repeat = True` and `exit=True`.
    
    In this scenario where `exit=True` is set, program will exit out at the first timeout as shown in the example below. Also note that it will exit the program in unsafe mode, that is no cleanup will be performed, if any files are open, they **might** be left open after the program has exited. 
    
    Also note, that it override's `repeat=True` and exit outs on the first occurrence of timeout.

```python
import faulthandler


def MySample():
    import time
    time.sleep(6)
    return "Welcome to the Wonders of Python"


faulthandler.dump_traceback_later(2, repeat=True, exit=True)
txt = MySample()
print(txt)
faulthandler.cancel_dump_traceback_later()

```
**Output:**
```python
Timeout (0:00:02)!
Thread 0x00007fba82f6f540 (most recent call first):
  File "timeout_exit_true_1_example.py", line 6 in MySample
  File "timeout_exit_true_1_example.py", line 11 in <module>
```
    

### User Signal

`faulthandler.register(signal)`

1. It is not available for Windows.

```python
from threading import Thread
import faulthandler
import signal
import time


def do():
    time.sleep(10)


def main():
    threads = []
    for _ in range(1, 10):
        t = Thread(target=do, args=())
        t.start()
    for x in threads:
        x.join()


if __name__ == '__main__':
    faulthandler.register(signal.SIGUSR1)
    main()
```
**Output:**
```python
$:> python3 register_example.py &
[1] 29577

$:> kill -s USR1 29577

Thread 0x00007f3f26ffd700 (most recent call first):
  File "register_example.py", line 8 in do
  File "/usr/lib64/python3.6/threading.py", line 864 in run
  File "/usr/lib64/python3.6/threading.py", line 916 in _bootstrap_inner
  File "/usr/lib64/python3.6/threading.py", line 884 in _bootstrap

Thread 0x00007f3f277fe700 (most recent call first):
  File "register_example.py", line 8 in do
  File "/usr/lib64/python3.6/threading.py", line 864 in run
  File "/usr/lib64/python3.6/threading.py", line 916 in _bootstrap_inner
  File "/usr/lib64/python3.6/threading.py", line 884 in _bootstrap

Thread 0x00007f3f27fff700 (most recent call first):
  File "register_example.py", line 8 in do
  File "/usr/lib64/python3.6/threading.py", line 864 in run
  File "/usr/lib64/python3.6/threading.py", line 916 in _bootstrap_inner
  File "/usr/lib64/python3.6/threading.py", line 884 in _bootstrap

Thread 0x00007f3f3cd67700 (most recent call first):
  File "register_example.py", line 8 in do
  File "/usr/lib64/python3.6/threading.py", line 864 in run
  File "/usr/lib64/python3.6/threading.py", line 916 in _bootstrap_inner
  File "/usr/lib64/python3.6/threading.py", line 884 in _bootstrap

Thread 0x00007f3f3d568700 (most recent call first):
  File "register_example.py", line 8 in do
  File "/usr/lib64/python3.6/threading.py", line 864 in run
  File "/usr/lib64/python3.6/threading.py", line 916 in _bootstrap_inner
  File "/usr/lib64/python3.6/threading.py", line 884 in _bootstrap

Thread 0x00007f3f3dd69700 (most recent call first):
  File "register_example.py", line 8 in do
  File "/usr/lib64/python3.6/threading.py", line 864 in run
  File "/usr/lib64/python3.6/threading.py", line 916 in _bootstrap_inner
  File "/usr/lib64/python3.6/threading.py", line 884 in _bootstrap
```

### Issue with file descriptors


`enable()`, `dump_traceback_later()` and `register()` keep the file descriptor of their file argument. If the file is closed and its file descriptor is reused by a new file, or if `os.dup2()` is used to replace the file descriptor, the traceback will be written into a different file. Call these functions again each time that the file is replaced.

### Reference

- https://docs.python.org/3/library/faulthandler.html
- https://blog.ionelmc.ro/2013/06/05/python-debugging-tools/